In [ ]:
from Solver import Solver
from MPA.MPA import MPA
from Heuristic.Greedy import Greedy
from Heuristic.FixOpt import FixOpt
import random as r
import json
from IPython.display import clear_output
import pandas as pd
import tabulate

r.seed(14)

options = {}

# Uncomment if you are using a WLS Gurobi license
# and write your credentials in credentials.txt
# with open('credentials.txt') as f: 
#     data = f.read() 
# options = json.loads(data)

In [ ]:
def generate_instance(N_cardinality, M_cardinality,
                      min_execution_time = 1,max_execution_time = 100,
                      min_setup_time = 1,max_setup_time = 100):
    
    P = {}
    S = {}
    
    N = range(1, N_cardinality+1)
    M = range(1, M_cardinality+1)
    
    for i in M:
        for j in N:
            P[i,j] = r.randint(min_execution_time, max_execution_time)
            
    for i in M:
        for j in N:
            for k in N:
                S[i,j,k] = r.randint(min_setup_time, max_setup_time)
    
    return P,S

N_cardinality = 4
M_cardinality = 2

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality,
                                   min_execution_time = 1, max_execution_time = 5,
                                   min_setup_time = 1, max_setup_time = 5)

# Tabulate P_dict
P_list = list(P_dict.items())
print(tabulate.tabulate(P_list, headers=['(i, j)', 'Execution Time']))
# Tabulate S_dict
S_list = list(S_dict.items())
print(tabulate.tabulate(S_list, headers=['(i, j, k)', 'Setup Time']))


In [ ]:
N = range(1, N_cardinality+1)
M = range(1, M_cardinality+1)
N0 = [i for i in N]
N0.insert(0,0)

In [ ]:
s = Solver(execution_times = P_dict, setup_times = S_dict)
decision_variables,completion_times,maximum_makespan,assignments = s.solve(options=options)
# clear_output(wait=True)
for i in s.N:
    print(f'Job {i} completed at {completion_times[i]}s')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

In [ ]:
s = MPA(execution_times = P_dict, setup_times = S_dict, t_max=10800) #  max_time = 1800
decision_variables,maximum_makespan = s.solve(options=options)
for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')